# Goal is to define the 95% CI of accuracy for a Classifier

In [1]:
import numpy as np
from sklearn.utils import resample
from sklearn.metrics import accuracy_score

## 1. Simulate a vector of accuracy

We are creating a simulated true label vector with only label '1', as well as a model that will be correct 60% of the time (it will predict 1, 60% of the time and 0 otherwise). That's to compare with something for which we know the true value, but in practice you can replace the simuModel by your own model.

In [2]:
class bernoulliModel():
    """Simply a bernoulli generator with probability 'proba' and 'n' being equal to the vector length to 'fit'.
    In practice we don't need X_train, or X_test, but I put it here for ease when using a real model.
    One could also use the np.random.binomial function directly to generate the list of accuracy (with accuracy = False)
    """
    def __init__(self, proba):
        self.p = proba
        
    def fit(self, X_train=None, y_train=None):
        return None
    
    def predict(self, X_test):
        self.n = len(X_test)
        return np.random.binomial(1, self.p, self.n)

## 2. Evaluate CI with Bernoulli asymptotatic law?

See https://stats.stackexchange.com/questions/4756/confidence-interval-for-bernoulli-sampling

$$CI = \hat{p} \pm z_{1 - \alpha / 2} * \sqrt{\frac{\hat{p}(1-\hat{p})}{n}}$$

In [3]:
simu_model = bernoulliModel(0.6)

In [4]:
n = 2000
y_test = np.ones(n)
X_test = np.ones((n, 3))
yest_test = simu_model.predict(X_test)

In [5]:
phat = accuracy_score(y_test, yest_test)

In [6]:
phat

0.5895

In [7]:
SE = np.sqrt(phat * (1 - phat) / n)
CI = (phat - 1.96*SE, phat + 1.96*SE)

In [8]:
CI

(0.567940456595744, 0.6110595434042561)

## 3. Evaluate CI with bootstrap

In [9]:
values = np.ones((2000, 3))

In [20]:
def bootstrap_model(data, model, n_iterations=1000, metric=accuracy_score):
    ldata = len(data)
    n_size = int(ldata * 0.50)
    list_index = np.arange(ldata)
    # run bootstrap
    stats = list()
    for i in range(n_iterations):
        # prepare train and test sets
        train_idx = resample(list_index, n_samples=n_size)
        oob_idx = [x for x in list_index if x not in train_idx]
        train = data[train_idx]
        test = data[oob_idx]
        # fit model
        model.fit(train[:,:-1], train[:,-1])
        # evaluate model
        predictions = model.predict(test[:,:-1])
        score = accuracy_score(test[:,-1], predictions)
        stats.append(score)
    return stats

In [11]:
stats = bootstrap_model(values, bernoulliModel(0.6))

### 3.1 Using the percentiles

In [12]:
# confidence intervals
alpha = 0.95
p = ((1.0-alpha)/2.0) * 100
lower = max(0.0, np.percentile(stats, p))
p = (alpha+((1.0-alpha)/2.0)) * 100
upper = min(1.0, np.percentile(stats, p))

In [13]:
CI = (lower, upper)

In [14]:
CI

(0.5728314238952537, 0.6263512992020305)

### 3.2 using the SE

In [15]:
phat = np.mean(stats)

In [16]:
phat

0.5997481703025431

In [17]:
SE = np.std(stats, ddof=1) 
CI = (phat - 1.96*SE, phat + 1.96*SE)

In [18]:
SE

0.013973209970655919

In [19]:
CI

(0.5723606787600575, 0.6271356618450288)